In [6]:
from result_saver import SaverProvider
from Scratch import metadata_loader
from Scratch import find_closest_calib_jobs

import numpy as np
from tqdm import tqdm
from time import sleep
from datetime import datetime
import json
import os

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi


def decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name):
    provider = SaverProvider()

    state = "X" if XBASIS else "Z"
    state += LOGICAL
    print(f"state = {state}")


    # Load the metadata
    while True:
        try:
            md = metadata_loader(True, True)
            break
        except:
            sleep(5)
    md = md[md["job_status"] == "JobStatus.DONE"]
    md = md[md["code"] == "RepetitionCodeCircuit"]
    md = md[md["descr"] == 'subset RepCodes']
    md = md.dropna(subset=["rounds"])
    md = md[md["meas_level"] == 1]
    md['rounds'] = md['rounds'].astype(int)
    md['distance'] = md['distance'].astype(int)

    md = md[md["backend_name"] == DEVICE]
    md = md[md["logical"] == LOGICAL]
    md = md[md["xbasis"] == XBASIS]
    md = md[md["rounds"] == ROUNDS]

    md = md[:1]

    print("shape:", md.shape)

    DOUBLE_MSMT = False # to get also 03-16

    jobs_by_calibration_date = {}
    for index, row in md.iterrows():
        job_id = row['job_id']

        while True:
            try:
                _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
                break
            except:
                sleep(5)

        if calib_creation_date not in jobs_by_calibration_date.keys():
            jobs_by_calibration_date[calib_creation_date] = [job_id]
        else:
            jobs_by_calibration_date[calib_creation_date].append(job_id)

    print(jobs_by_calibration_date)
    print()
    print(f"num of calibrations: {len(jobs_by_calibration_date)}")
    print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")


    d = md["distance"].values[0]

    distances = np.arange(7, d+1, 4)
    distances = distances[::-1]

    
    HANDLE_OUTLIERS = True

    rel_error = 1
    _RESETS = False

    # KDE BANDWIDTHS
    lin = [0.6, 1.2, 1]
    num_points = 20
    # lin = [0.1, 0.7, 1]
    # num_points = 7
    bandwidths = np.linspace(lin[0], lin[1], lin[2])


    for calib_date in jobs_by_calibration_date.keys():
        # get the noise dict of that date
        noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

        # get the KDE of that date

        while True:
            try:
                all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=False)
                break
            except:
                sleep(5)

        kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        while True:
            try:
                all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)
                break
            except:
                sleep(5)

        for qubit, data in all_memories_PS.items():
            lens = (data['mmr_0'].shape[0], data['mmr_1'].shape[0])
            min_len = min(lens)
            all_memories_PS[qubit]['mmr_0'] = data['mmr_0'][:min_len]
            all_memories_PS[qubit]['mmr_1'] = data['mmr_1'][:min_len]

        for qubit, data in all_memories_PS.items():
            print(qubit, len(data['mmr_0']), len(data['mmr_1']))
            
        kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        # Get the mean msmt errors
        p_soft_mean = 0
        p_hard_mean = 0
        for key, value in msmt_err_dict_PS.items():
            p_soft_mean += value['p_soft']
            p_hard_mean += value['p_hard']
        p_soft_mean /= len(msmt_err_dict_PS)
        p_hard_mean /= len(msmt_err_dict_PS)
        print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)

        for i in range(127):
            print(f"qubit {i}: {kde_dict[i].bestBandwidth}", end=' ')
        print("\nPS", end=' ')
        for i in range(127):
            print(f"qubit {i}: {kde_dict_PS[i].bestBandwidth}", end=' ')

        # Retrieve the memories
        memories = []
        for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
            d = md[md["job_id"] == job_id]["distance"].values[0] 
            T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

            # Get the job
            job = provider.retrieve_job(job_id)
            memory = job.result().get_memory()
            memories.append(memory)

        # Stack the memories vertically
        big_memory = np.vstack(memories)

        # Get the layout of the last job (same as previous)
        layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
        link_qubits = list(layout_des['link_qubit'].values())
        code_qubits = list(layout_des['code_qubit'].values())

        # Get the pSoft and countMat matrices
        big_layout = link_qubits + code_qubits
        inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

        print(f"Starting to get pSoft and countMat at {datetime.now()}")
        pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                    handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
        pSoft_PS, countMat_PS = csi.iqConvertor(big_memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                        handleOutliers = HANDLE_OUTLIERS)
        
        print(f"Starting to get pSoftG at {datetime.now()}")
        countMatG, pSoftG = gaussianIQConvertor(big_memory, inverted_q_map, gmm_dict)

        assert pSoft.shape[0] == pSoft_PS.shape[0] == pSoftG.shape[0], "Different number of shots for each decoding method"
        NB_SHOTS = pSoft.shape[0]

        # Subsample decoding
        for D_NEW in tqdm(distances):
            subsets = generate_subsets_with_center(d, D_NEW)

            err_s_K_mean = []
            err_h_K_mean = []
            err_s_KPS_mean = []
            err_h_KPS_mean = []
            
            err_s_K_DYN_indiv = []
            err_s_K_DYN_mean = []
            err_s_KPS_DYN_indiv = []
            err_s_KPS_DYN_mean = []
            for subset in (subsets):             
                cols_to_keep = get_cols_to_keep(subset, T, d)

                # Get the subset of pSoft and countMat
                pSoft_sub = pSoft[:, cols_to_keep]
                pSoft_PS_sub = pSoft_PS[:, cols_to_keep]
                # pSoftG_sub = pSoftG[:, cols_to_keep]
                countMat_sub = countMat[:, cols_to_keep]
                countMat_PS_sub = countMat_PS[:, cols_to_keep]
                # countMatG_sub = countMatG[:, cols_to_keep]

                # Get the layout
                layout = get_subsample_layout(subset, link_qubits, code_qubits)
                qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

                # Get the noise avgs
                avgs = get_avgs_from_dict(noise_dict, layout)
                noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
                readout = avgs["readout"]
                noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
                #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

                # Get the msmt err avgs
                p_hard_0 = np.mean([msmt_err_dict_PS[qubit]['p_hard_0'] for qubit in layout])
                p_hard_1 = np.mean([msmt_err_dict_PS[qubit]['p_hard_1'] for qubit in layout])
                p_soft_0 = np.mean([msmt_err_dict_PS[qubit]['p_soft_0'] for qubit in layout])
                p_soft_1 = np.mean([msmt_err_dict_PS[qubit]['p_soft_1'] for qubit in layout])

                msmt_err_dict_mean = {qubit:{'p_hard_0':p_hard_0, 'p_hard_1':p_hard_1, 'p_soft_0':p_soft_0, 'p_soft_1':p_soft_1} for qubit in msmt_err_dict_PS.keys()}
                # msmt_err_dict_mean = {qubit:{'p_hard_0': p_hard_mean, 'p_hard_1': p_hard_mean, 'p_soft_0': p_soft_mean, 'p_soft_1': p_soft_mean} for qubit in msmt_err_dict_PS.keys()}



                # Stim models
                subsampling = (D_NEW != d)

                code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

                code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

                code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()

                new_noise_list = noise_list.copy()
                new_noise_list[5] = 0
                code_indiv_for_dynamic = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                                msmt_err_dict=msmt_err_dict_PS)
                model_indiv_for_dynamic = code_indiv_for_dynamic.circuits[LOGICAL].detector_error_model()
                code_mean_for_dynamic = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                                msmt_err_dict=None)
                model_mean_for_dynamic = code_mean_for_dynamic.circuits[LOGICAL].detector_error_model()
                

                # Decoding

                ### Dynamic 
                res_s_K_DYN_indiv = csi.decodeConvertorDynamicAll(model_indiv_for_dynamic, countMat_sub, pSoft_sub, 
                                                                  msmt_err_dict_PS, qubit_mapping, T,
                                                                  int(LOGICAL), _RESETS, decode_hard=False)
                res_s_K_DYN_mean = csi.decodeConvertorDynamicAll(model_mean_for_dynamic, countMat_sub, pSoft_sub,
                                                                  msmt_err_dict_mean, qubit_mapping, T,
                                                                  int(LOGICAL), _RESETS, decode_hard=False)
                
                res_s_KPS_DYN_indiv = csi.decodeConvertorDynamicAll(model_indiv_for_dynamic, countMat_PS_sub, pSoft_PS_sub,
                                                                  msmt_err_dict_PS, qubit_mapping, T,
                                                                  int(LOGICAL), _RESETS, decode_hard=False)
                res_s_KPS_DYN_mean = csi.decodeConvertorDynamicAll(model_mean_for_dynamic, countMat_PS_sub, pSoft_PS_sub,
                                                                  msmt_err_dict_mean, qubit_mapping, T,
                                                                  int(LOGICAL), _RESETS, decode_hard=False)

                
                ### KDE
                res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)

                
                
                ### KDE PS
                res_s_KPS_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_h_KPS_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                                        int(LOGICAL), _RESETS, decode_hard=True)
                
                # Append the results
                err_s_K_mean.append(res_s_K_mean.num_errors)
                err_h_K_mean.append(res_h_K_mean.num_errors)
                err_s_KPS_mean.append(res_s_KPS_mean.num_errors)
                err_h_KPS_mean.append(res_h_KPS_mean.num_errors)

                err_s_K_DYN_indiv.append(res_s_K_DYN_indiv.num_errors)
                err_s_K_DYN_mean.append(res_s_K_DYN_mean.num_errors)
                err_s_KPS_DYN_indiv.append(res_s_KPS_DYN_indiv.num_errors)
                err_s_KPS_DYN_mean.append(res_s_KPS_DYN_mean.num_errors)
    


            # Save the results
            json_KDE = {
                "decoding": "kde",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_K_mean,
                    "hard_mean": err_h_K_mean,
                    "soft_DYN_indiv": err_s_K_DYN_indiv,
                    "soft_DYN_mean": err_s_K_DYN_mean,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_K_mean),
                    "hard_mean": np.mean(err_h_K_mean),
                    "soft_DYN_indiv": np.mean(err_s_K_DYN_indiv),
                    "soft_DYN_mean": np.mean(err_s_K_DYN_mean),
                },   
                "additional_info": {
                    "rel_error": rel_error,
                    "bandwidth_linspace": lin,
                    "num_points_bandwidths": num_points,
                    "p_hard_mean_0" : p_hard_0,
                    "p_hard_mean_1" : p_hard_1,
                    "p_soft_mean_0" : p_soft_0,
                    "p_soft_mean_1" : p_soft_1,
                },        
            }

            json_KDE_PS = {
                "decoding": "kde_PS",
                "d_new": str(D_NEW),
                "error_list_dict": {
                    "soft_mean": err_s_KPS_mean,
                    "hard_mean": err_h_KPS_mean,
                    "soft_DYN_indiv": err_s_KPS_DYN_indiv,
                    "soft_DYN_mean": err_s_KPS_DYN_mean,
                },
                "error_mean_dict": {
                    "soft_mean": np.mean(err_s_KPS_mean),
                    "hard_mean": np.mean(err_h_KPS_mean),
                    "soft_DYN_indiv": np.mean(err_s_KPS_DYN_indiv),
                    "soft_DYN_mean": np.mean(err_s_KPS_DYN_mean),
                },   
                "additional_info": {
                },
            }

            if not os.path.exists(file_name):
                data = {}
            else:
                with open(file_name, "r") as f:
                    data = json.load(f)

            if job_id not in data.keys():
                data[job_id] = [{'totshots': NB_SHOTS}, json_KDE, json_KDE_PS]
            else:
                data[job_id].append(json_KDE)
                data[job_id].append(json_KDE_PS)
            
            with open(file_name, "w") as f:
                json.dump(data, f, indent=4)

            # rm the hard decoder after first run!

In [7]:
logicals = ['0']
xbasis = [True, False]
rounds = [50]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'../results/result_day5/KDE_SAME_SIZE_{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
shape: (1, 18)
{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g']}

num of calibrations: 1
num of jobs per calibration: [1]
21:47:34 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
21:47:34 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
21:47:34 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbr

Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


Starting to get pSoft and countMat at 2024-04-11 21:48:31.836518
Starting to get pSoft_PS and countMat_PS at 2024-04-11 21:49:02.709864
Starting to get pSoftG at 2024-04-11 21:49:22.659489


100%|██████████| 12/12 [03:29<00:00, 17.42s/it]


state = Z0
shape: (1, 18)
{datetime.datetime(2024, 3, 23, 13, 5, 51, 979000, tzinfo=datetime.timezone.utc): ['cqzd7adk5z70008j0810']}

num of calibrations: 1
num of jobs per calibration: [1]
21:53:01 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
21:53:01 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
21:53:01 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbr

Retrieving jobs of 2024-03-23 13:05:51.979000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


Starting to get pSoft and countMat at 2024-04-11 21:54:21.547790
Starting to get pSoft_PS and countMat_PS at 2024-04-11 21:54:52.650217


In [3]:
all_memories_PS

NameError: name 'all_memories_PS' is not defined

In [ ]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [75]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)